In [1]:
import MDP_function as mf
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import scipy.stats as stats
import random as rnd
%matplotlib inline

In [2]:
# read in all original data set
data_file = "MDP_Original_data.csv"
original_data = pd.read_csv(data_file)

In [3]:
# get names of all columns in original dataset
original_data.columns.tolist()

['student',
 'currProb',
 'course',
 'session',
 'priorTutorAction',
 'reward',
 'Interaction',
 'hintCount',
 'TotalTime',
 'TotalPSTime',
 'TotalWETime',
 'avgstepTime',
 'avgstepTimePS',
 'stepTimeDeviation',
 'symbolicRepresentationCount',
 'englishSymbolicSwitchCount',
 'Level',
 'probDiff',
 'difficultProblemCountSolved',
 'difficultProblemCountWE',
 'easyProblemCountSolved',
 'easyProblemCountWE',
 'probAlternate',
 'easyProbAlternate',
 'RuleTypesCount',
 'UseCount',
 'PrepCount',
 'MorphCount',
 'OptionalCount',
 'NewLevel',
 'SolvedPSInLevel',
 'SeenWEinLevel',
 'probIndexinLevel',
 'probIndexPSinLevel',
 'InterfaceErrorCount',
 'RightApp',
 'WrongApp',
 'WrongSemanticsApp',
 'WrongSyntaxApp',
 'PrightAppRatio',
 'RrightAppRatio',
 'F1Score',
 'FDActionCount',
 'BDActionCount',
 'DirectProofActionCount',
 'InDirectProofActionCount',
 'actionCount',
 'UseWindowInfo',
 'NonPSelements',
 'AppCount',
 'AppRatio',
 'hintRatio',
 'BlankRatio',
 'HoverHintCount',
 'SystemInfoHintCou

In [4]:
# select all features from the original dataset and get features names as feature space indexes
feature_data = original_data.loc[:, 'Interaction':'CurrPro_medianProbTime']
feature_space = feature_data.columns.tolist()
print feature_space

['Interaction', 'hintCount', 'TotalTime', 'TotalPSTime', 'TotalWETime', 'avgstepTime', 'avgstepTimePS', 'stepTimeDeviation', 'symbolicRepresentationCount', 'englishSymbolicSwitchCount', 'Level', 'probDiff', 'difficultProblemCountSolved', 'difficultProblemCountWE', 'easyProblemCountSolved', 'easyProblemCountWE', 'probAlternate', 'easyProbAlternate', 'RuleTypesCount', 'UseCount', 'PrepCount', 'MorphCount', 'OptionalCount', 'NewLevel', 'SolvedPSInLevel', 'SeenWEinLevel', 'probIndexinLevel', 'probIndexPSinLevel', 'InterfaceErrorCount', 'RightApp', 'WrongApp', 'WrongSemanticsApp', 'WrongSyntaxApp', 'PrightAppRatio', 'RrightAppRatio', 'F1Score', 'FDActionCount', 'BDActionCount', 'DirectProofActionCount', 'InDirectProofActionCount', 'actionCount', 'UseWindowInfo', 'NonPSelements', 'AppCount', 'AppRatio', 'hintRatio', 'BlankRatio', 'HoverHintCount', 'SystemInfoHintCount', 'NextStepClickCountWE', 'PreviousStepClickCountWE', 'deletedApp', 'ruleScoreMP', 'ruleScoreDS', 'ruleScoreSIMP', 'ruleScore

In [5]:
# initialize parameters and data structures for correlation-based feature selection algorithm
MAX_NUM_OF_FEATURES = 2
ECR_list = list()
optimal_feature_set = list()

In [6]:
def feature_discretization(feature_data, maxLevel=10): 
    # discretize continuous feature values into integers of no more than max levels
    isFloat = any(map(lambda x: isinstance(x, float), feature_data)) # check if it contain float type
    if not isFloat:
        isOverLevel = len(feature_data.unique())>maxLevel # check if it is within max levels
    if isFloat or isOverLevel: # discretize and reduce levels using median
        median = feature_data.median()
        feature_vals = map(lambda x: 0 if x<=median else 1, feature_data)
        feature_data = pd.Series(feature_vals, dtype=int)
    return feature_data

In [7]:
# discretization feature values by median
all_data_discretized = original_data.loc[:, "student":"reward"]
for i, ft in enumerate(feature_space):
    ft_data = original_data.loc[:, ft]
    all_data_discretized[ft] = feature_discretization(ft_data)
all_data_discretized.describe()

,session,reward,Interaction,hintCount,TotalTime,TotalPSTime,TotalWETime,avgstepTime,avgstepTimePS,stepTimeDeviation,...,cumul_NextStepClickCountWE,cumul_PreviousStepClickCountWE,cumul_deletedApp,CurrPro_NumProbRule,CurrPro_avgProbTime,CurrPro_avgProbTimePS,CurrPro_avgProbTimeDeviationPS,CurrPro_avgProbTimeWE,CurrPro_avgProbTimeDeviationWE,CurrPro_medianProbTime
count,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,...,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000
mean,2.872611,-0.357487,0.493631,0.199955,0.500000,0.500000,0.318926,0.500000,0.203822,0.351911,...,0.421747,0.197452,0.453822,0.424704,0.476115,0.495905,0.494313,0.491811,0.468835,0.475205
std,2.514063,34.131956,0.500016,0.400011,0.500057,0.500057,0.466113,0.500057,0.402884,0.477621,...,0.493895,0.398122,0.497920,0.494354,0.499486,0.500040,0.500025,0.499990,0.499085,0.499442
min,1.000000,-305.714286,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.500000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,...,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,19.000000,200.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# initialization to find the best feature with max ECR
for i, ft in enumerate(feature_space):
    #print i
    selected_feature = [ft]
    # try:
    ECR_list.append(mf.compute_ECR(all_data_discretized, selected_feature))
#     except:
#         ECR_list.append(0.0)
print ECR_list

[0.86239839583987621, 0.68601588332982288, 0.82512603395470419, 3.1078311846207813, 9.6752234066668095, 0.38084556096440481, 0.90448896681597413, 0.0, 0.0, 0.0, 3.9820569642858041, 0.0, 27.495904195466874, 16.881755706664304, 3.0313199185550719, 0.0, 1.3036139603639174, 1.3036139603639174, 0.0, 0.39974947264589067, 2.8473124450876739, 0.0, 1.6271295216631378, 0.0, 0.0, 10.177936326858347, 24.886652411147228, 9.2373172826207117, 0.20207066883902353, 0.0, 4.6822168974818279, 3.5457651945044271, 4.6865060409625823, 0.58680892494288417, 0.037998665510776734, 1.263903559175775, 1.8610273161470614, 0.0, 0.90456747959581352, 0.0, 1.8609505317973556, 0.0, 0.0, 0.77852573237224587, 0.0, 0.81249222916645758, 0.0, 0.88290329496840891, 0.0, 10.173464801031713, 1.6260057021458632, 0.0, 3.0667556841179247, 2.5467379811527131, 0.0032785153328480673, 0.0, 0.0, 6.9231378481716392, 6.6803934627589197, 3.6557811292624907, 5.8312445115843889, 0.0, 2.8348540537288511, 0.0, 2.6476628433014393, 1.62770654387

/Users/Jerry/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
optimal_feature_set.append(feature_space[ECR_list.index(max(ECR_list))])
print(optimal_feature_set)

['difficultProblemCountSolved']


In [10]:
max(ECR_list)

27.495904195466874

In [11]:
def compute_correlation(dataset, feature_set, feature):
#     corr_sum = 0
#     for ft in feature_set:
#         corr, p_val = stats.pearsonr(dataset[ft], dataset[feature])
#         corr_sum += corr
#     return corr_sum
    return rnd.random()

In [12]:
# feature selection iterations
while (len(optimal_feature_set) < MAX_NUM_OF_FEATURES):
    #print len(optimal_feature_set)
    corr_list = list()
    remain_feature_space = list(set(feature_space) - set(optimal_feature_set))
    for i,ft in enumerate(remain_feature_space):
        corr_list.append([ft, compute_correlation(all_data_discretized, optimal_feature_set, ft)])
    topK = 5
    top_features = map(lambda x: x[0], sorted(corr_list, key=lambda x: x[1], reverse=False)[:topK])
    ECR_list = list()
    for i, ft in enumerate(top_features):
        select_feature = list(optimal_feature_set)
        select_feature.append(ft)
        ECR_list.append([ft, mf.compute_ECR(all_data_discretized, selected_feature)])
        print ECR_list
    best_next_feature_and_ECR = sorted(ECR_list, key=lambda x: x[1], reverse=True)[0]
    best_next_feature, bestECR = best_next_feature_and_ECR[0], best_next_feature_and_ECR[1]
    optimal_feature_set.append(best_next_feature)
    print best_next_feature_and_ECR

[['BlankRatio', 3.0005676909743726]]
[['BlankRatio', 3.0005676909743726], ['NextStepClickCountWE', 3.0005676909743726]]
[['BlankRatio', 3.0005676909743726], ['NextStepClickCountWE', 3.0005676909743726], ['TotalWETime', 3.0005676909743726]]
[['BlankRatio', 3.0005676909743726], ['NextStepClickCountWE', 3.0005676909743726], ['TotalWETime', 3.0005676909743726], ['SeenWEinLevel', 3.0005676909743726]]
[['BlankRatio', 3.0005676909743726], ['NextStepClickCountWE', 3.0005676909743726], ['TotalWETime', 3.0005676909743726], ['SeenWEinLevel', 3.0005676909743726], ['ruleScoreTAUT', 3.0005676909743726]]
['BlankRatio', 3.0005676909743726]


/Users/Jerry/anaconda/lib/python2.7/site-packages/scipy/stats/stats.py:3028: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [13]:
optimal_feature_set

['difficultProblemCountSolved', 'BlankRatio']

In [15]:
mf.induce_policy_MDP2(all_data_discretized, optimal_feature_set)

Policy: 
state -> action, value-function
0:1 -> PS, 34.2742383682
0:0 -> WE, 29.8142209888
1:0 -> PS, 51.1421557847
1:1 -> PS, 67.1592115291
ECR value: 33.4743439469


33.474343946852628